In [99]:
# %reload_ext autoreload
# %autoreload 2
from text_preprocessing import *

In [243]:
print('%s and %s' %('hi','hi2'))

hi and hi2


In [21]:
import wikipediaapi
import wikitextparser as wtp
import requests
import urllib.parse
from opencc import OpenCC
import json


wiki = wikipediaapi.Wikipedia('zh')

def s2t(text):
    cc = OpenCC('s2t')
    return cc.convert(text)

def get_sections_data(title):
    page = wiki.page(title)
    results = get_image_data(title)
    sections = [{'title':None, 'level':0, 'content':page.summary, 'main_image_url':get_wiki_main_image(title)}]
    for section in page.sections:
        sections.append({'title':section.title, 'level':1,'content':section.text})
        for subsection in section.sections:
            sections.append({'title':subsection.title, 'level':2,'content':subsection.text})
            for subsubsection in subsection.sections:
                sections.append({'title':subsubsection.title,'level':3,'content':subsubsection.text})
    
    
    if len(sections) == len(results):
        for idx,result in enumerate(results):
            sections[idx]['images'] = result['images']
    elif len(sections) != len(results):
        result_titles = [clean_str(result['title']) for result in results]
        for section in sections:
            try:
                idx = result_titles.index(clean_str(section['title']))
            except:
                continue
            section['images'] = results[idx]['images']
    return sections
        
#     return remove_repeated_img_from_sections(sections)

def clean_str(string):
    if not string:
        return string
    else:
        return s2t(string.strip())

def get_image_data(title):
    r = requests.get(
        'https://zh.wikipedia.org/w/api.php',
        params={
            'action': 'parse',
            'page': wiki.page(title).displaytitle,
            'contentmodel': 'wikitext',
            'prop': 'wikitext',
            'format': 'json'
        }
    )

    data = wtp.parse(r.json()['parse']['wikitext']['*'])
    results = []
    for section in data.sections:
    #     print(f"section title: {section.title}")
        images = [
            {   "metadata": t.text.split("|"),
                "title": t.title[5:],
                "api_url": f"https://zh.wikipedia.org/w/api.php?action=query&titles={urllib.parse.quote_plus(t.title)}&prop=imageinfo&iiprop=url&format=json"
            }
            for t in wtp.parse(section.contents).wikilinks
            if t.title.startswith("File")
        ]
#         print(section.title)

        for image in images:
            number = list(requests.get(image["api_url"]).json()['query']['pages'].keys())[0]
            image["url"] = requests.get(image["api_url"]).json()[
                "query"]["pages"][number]["imageinfo"][0]["url"]
#             print(requests.get(image["api_url"]).json())
        results.append({
            "title": section.title,
            "images": images
        })
    
    return results

def get_wiki_main_image(title):
    url = 'https://zh.wikipedia.org/w/api.php'
    data = {
        'action' :'query',
        'format' : 'json',
        'formatversion' : 2,
        'prop' : 'pageimages|pageterms',
        'piprop' : 'original',
        'titles' : wiki.page(title).displaytitle
    }
    try:
        response = requests.get(url, data)
        json_data = json.loads(response.text)
        return json_data['query']['pages'][0]['original']['source'] if len(json_data['query']['pages']) >0 else 'Not found'
    except:
        return None
    
def remove_repeated_img_from_sections(sections):
    sections_copy = sections.copy()
    for idx1,section in enumerate(sections_copy):
        try:
            if section['level'] == 1:
                images_list = section['images']
                for idx2,section2 in enumerate(sections_copy):
                    if idx2 > idx1 and section2['level'] > 1:
                        images_list = [x for x in images_list if x not in section2['images']]
                    if section2['level'] == 1:
                        sections_copy[idx1]['images'] = images_list
                        break
        except Exception as e:
            print("Some errors occured for removing repeated images.")
            print(e)
    for idx1,section in enumerate(sections_copy):
        try:
            if section['level'] == 2:
                images_list = section['images']
                for idx2,section2 in enumerate(sections_copy):
                    if idx2 > idx1 and section2['level'] > 2:
                        images_list = [x for x in images_list if x not in section2['images']]
                    if section2['level'] == 2:
                        sections_copy[idx1]['images'] = images_list
                        break
        except Exception as e:
            print("Some errors occured for removing repeated images.")
            print(e)
    return sections_copy



In [246]:
data = get_image_data('希特拉')

In [22]:
remove_repeated_img_from_sections(sections)

Some errors occured for removing repeated images.
'images'


[{'title': None,
  'level': 0,
  'content': '阿道夫·希特勒（德語：Adolf Hitler，1889年4月20日－1945年4月30日），德國政治人物，前纳粹党领袖，1933年至1945年担任德国总理，1934年至1945年亦任元首。其于1939年9月发动波兰战役，导致第二次世界大战在欧洲爆发，并为纳粹大屠杀的主要策劃者、发动者之一。證據顯示希特勒知曉並下達了對猶太人的屠殺命令。根據阿道夫·艾希曼、海因里希·希姆莱和約瑟夫·戈培爾等人的陳述，希特勒本人策劃了對猶太人的大屠殺；而希特勒本人的陳述亦顯示其意圖屠盡猶太民族。希特勒生于时属奥匈帝国的奥地利地区，長於林茨。1913年其迁往德国，并于第一次世界大战中在德国陆军服役並受勳。1919年希特勒加入纳粹党前身德国工人党，并于1921年成为纳粹党领袖。1923年他在慕尼黑發动啤酒馆政变，试图夺取权力，但最终失败并被监禁。在监禁期间希特勒撰写了其自传及政治宣言《我的奮鬥》的第一册。1924年被释放后，希特勒對凡爾赛条约进行批判，宣扬泛日耳曼主义、反犹太主义和反共主义，以其个人魅力、演说才能及政治宣传获得了广泛的民众支持。同时他频繁宣称国际资本主义及共产主义为犹太人反德意志民族的阴谋。1933年，纳粹党成为德国国会第一大党，1月30日希特勒被任命为德国总理。此后其执政联盟再次在选举中获胜，并在国会中通过《授权法》，逐渐将德国转变为一党专制、纳粹、极权及独裁统治的纳粹德国。希特勒借口抵御一战后由英国和法国主导的不公国际秩序，试图将犹太人从德国清除，建立其理想秩序。在其当政的前六年内，德国迅速自大萧条中复苏，打破一战后欧洲各国对其作出的各种限制，吞并数个德意志民族所居住的他国领土，由此获得大多数德意志民族人民的支持。\n希特勒试图为德意志民族获取在其他民族之上的特别“生存空间”，在外交上主张施行侵略与吞并，是第二次世界大战在欧洲爆发的首要原因。他领导大规模武装扩军，并于1939年9月1日入侵波兰，打响第二次世界大战并使得英国和法国对德国宣战。1941年6月22日，希特勒发动巴巴罗萨行动入侵苏联。至1941年年末，德国及其欧洲轴心国盟友已侵略占领欧洲和北非大部分地区。后来与苏联战事的不利局面及美国的加入导致德国由攻转守，并屡遭战略失败。在战争尾声阶段，1945年柏林战役期间，希特勒与其

In [9]:
sections = get_sections_data('希特拉')
sections

[{'title': None,
  'level': 0,
  'content': '阿道夫·希特勒（德語：Adolf Hitler，1889年4月20日－1945年4月30日），德國政治人物，前纳粹党领袖，1933年至1945年担任德国总理，1934年至1945年亦任元首。其于1939年9月发动波兰战役，导致第二次世界大战在欧洲爆发，并为纳粹大屠杀的主要策劃者、发动者之一。證據顯示希特勒知曉並下達了對猶太人的屠殺命令。根據阿道夫·艾希曼、海因里希·希姆莱和約瑟夫·戈培爾等人的陳述，希特勒本人策劃了對猶太人的大屠殺；而希特勒本人的陳述亦顯示其意圖屠盡猶太民族。希特勒生于时属奥匈帝国的奥地利地区，長於林茨。1913年其迁往德国，并于第一次世界大战中在德国陆军服役並受勳。1919年希特勒加入纳粹党前身德国工人党，并于1921年成为纳粹党领袖。1923年他在慕尼黑發动啤酒馆政变，试图夺取权力，但最终失败并被监禁。在监禁期间希特勒撰写了其自传及政治宣言《我的奮鬥》的第一册。1924年被释放后，希特勒對凡爾赛条约进行批判，宣扬泛日耳曼主义、反犹太主义和反共主义，以其个人魅力、演说才能及政治宣传获得了广泛的民众支持。同时他频繁宣称国际资本主义及共产主义为犹太人反德意志民族的阴谋。1933年，纳粹党成为德国国会第一大党，1月30日希特勒被任命为德国总理。此后其执政联盟再次在选举中获胜，并在国会中通过《授权法》，逐渐将德国转变为一党专制、纳粹、极权及独裁统治的纳粹德国。希特勒借口抵御一战后由英国和法国主导的不公国际秩序，试图将犹太人从德国清除，建立其理想秩序。在其当政的前六年内，德国迅速自大萧条中复苏，打破一战后欧洲各国对其作出的各种限制，吞并数个德意志民族所居住的他国领土，由此获得大多数德意志民族人民的支持。\n希特勒试图为德意志民族获取在其他民族之上的特别“生存空间”，在外交上主张施行侵略与吞并，是第二次世界大战在欧洲爆发的首要原因。他领导大规模武装扩军，并于1939年9月1日入侵波兰，打响第二次世界大战并使得英国和法国对德国宣战。1941年6月22日，希特勒发动巴巴罗萨行动入侵苏联。至1941年年末，德国及其欧洲轴心国盟友已侵略占领欧洲和北非大部分地区。后来与苏联战事的不利局面及美国的加入导致德国由攻转守，并屡遭战略失败。在战争尾声阶段，1945年柏林战役期间，希特勒与其

In [20]:
for section in sections:
    print(section['title'])
    if 'images' in section.keys():
        for image in section['images']:
            print(image['url'])


None
https://upload.wikimedia.org/wikipedia/commons/c/cf/Flag_of_the_NSDAP_%281920%E2%80%931945%29.svg
早期
https://upload.wikimedia.org/wikipedia/commons/e/e8/Bundesarchiv_Bild_183-1989-0322-506%2C_Adolf_Hitler%2C_Kinderbild_retouched.jpg
https://upload.wikimedia.org/wikipedia/commons/4/44/Adolf_Hitler_as_a_child.jpg
https://upload.wikimedia.org/wikipedia/commons/e/e5/Adolf_Hitler_-_Wien_Oper.jpg
https://upload.wikimedia.org/wikipedia/commons/f/f2/Hitler_WWI.jpg
家世
https://upload.wikimedia.org/wikipedia/commons/e/e8/Bundesarchiv_Bild_183-1989-0322-506%2C_Adolf_Hitler%2C_Kinderbild_retouched.jpg
https://upload.wikimedia.org/wikipedia/commons/4/44/Adolf_Hitler_as_a_child.jpg
童年
浪迹维也纳
https://upload.wikimedia.org/wikipedia/commons/e/e5/Adolf_Hitler_-_Wien_Oper.jpg
https://upload.wikimedia.org/wikipedia/commons/f/f2/Hitler_WWI.jpg
第一次世界大戰
進入政界
https://upload.wikimedia.org/wikipedia/commons/8/85/Hitler%27s_DAP_membership_card.png
https://upload.wikimedia.org/wikipedia/commons/f/fc/Bundesarch

In [237]:
get_image_data('希特勒')

[{'title': None,
  'images': [{'metadata': ['20px', 'border', 'link='],
    'title': 'Flag of the NSDAP (1920–1945).svg',
    'api_url': 'https://zh.wikipedia.org/w/api.php?action=query&titles=File%3AFlag+of+the+NSDAP+%281920%E2%80%931945%29.svg&prop=imageinfo&iiprop=url&format=json',
    'url': 'https://upload.wikimedia.org/wikipedia/commons/c/cf/Flag_of_the_NSDAP_%281920%E2%80%931945%29.svg'}]},
 {'title': ' 早期 ',
  'images': [{'metadata': ['thumb', '幼年時期的希特勒'],
    'title': 'Bundesarchiv Bild 183-1989-0322-506, Adolf Hitler, Kinderbild retouched.jpg',
    'api_url': 'https://zh.wikipedia.org/w/api.php?action=query&titles=File%3ABundesarchiv+Bild+183-1989-0322-506%2C+Adolf+Hitler%2C+Kinderbild+retouched.jpg&prop=imageinfo&iiprop=url&format=json',
    'url': 'https://upload.wikimedia.org/wikipedia/commons/e/e8/Bundesarchiv_Bild_183-1989-0322-506%2C_Adolf_Hitler%2C_Kinderbild_retouched.jpg'},
   {'metadata': ['thumb', 'upright=0.7', 'left', '青少年時期的希特勒'],
    'title': 'Adolf Hitler as a

In [234]:
for section in sections:
    print(section['title'])
    if 'images' in section:
        for image in section['images']:
            print(image['url'])

None
https://upload.wikimedia.org/wikipedia/commons/c/cf/Flag_of_the_NSDAP_%281920%E2%80%931945%29.svg
早期
https://upload.wikimedia.org/wikipedia/commons/e/e8/Bundesarchiv_Bild_183-1989-0322-506%2C_Adolf_Hitler%2C_Kinderbild_retouched.jpg
https://upload.wikimedia.org/wikipedia/commons/4/44/Adolf_Hitler_as_a_child.jpg
https://upload.wikimedia.org/wikipedia/commons/e/e5/Adolf_Hitler_-_Wien_Oper.jpg
https://upload.wikimedia.org/wikipedia/commons/f/f2/Hitler_WWI.jpg
家世
https://upload.wikimedia.org/wikipedia/commons/e/e8/Bundesarchiv_Bild_183-1989-0322-506%2C_Adolf_Hitler%2C_Kinderbild_retouched.jpg
https://upload.wikimedia.org/wikipedia/commons/4/44/Adolf_Hitler_as_a_child.jpg
童年
浪迹维也纳
https://upload.wikimedia.org/wikipedia/commons/e/e5/Adolf_Hitler_-_Wien_Oper.jpg
https://upload.wikimedia.org/wikipedia/commons/f/f2/Hitler_WWI.jpg
第一次世界大戰
https://upload.wikimedia.org/wikipedia/commons/8/85/Hitler%27s_DAP_membership_card.png
https://upload.wikimedia.org/wikipedia/commons/f/fc/Bundesarchiv_Bi

In [231]:
textlist = []
section_title = ''
for subsection in sections:
    if subsection['level'] == 1:
        print('-'*50)
        if section_title:
            print(section_title)
        else:
            print("Summary\n")
        text = s2t('\n'.join(textlist))
        try:
            print('subtitles:')
            print(get_subtitles_from_textfile(text.strip('\n')))
        except:
            print("Error making subtitles")
        section_title = subsection['title']
        textlist = []
    textlist.append(subsection['content'])
    
        

--------------------------------------------------
Summary

subtitles:
['阿道夫·希特勒', '德國政治人物', '前納粹黨領袖', '1933年至1945年擔任德國總理', '1934年至1945年亦任元首', '其於1939年9月發動波蘭戰役', '導致第二次世界大戰在歐洲爆發', '併爲納粹大屠殺的主要策劃者、發動者之一', '證據顯示希特勒知曉並下達了對猶太人的屠殺命令', '根據阿道夫·艾希曼、海因裏希·希姆萊和約瑟夫·戈培爾等人的陳述', '希特勒本人策劃了對猶太人的大屠殺', '而希特勒本人的陳述亦顯示其意圖屠盡猶太民族', '希特勒生於時屬奧匈帝國的奧地利地區', '長於林茨', '1913年其遷往德國', '並於第一次世界大戰中在德國陸軍服役並受勳', '1919年希特勒加入納粹黨前身德國工人黨', '並於1921年成爲納粹黨領袖', '1923年他在慕尼黑發動啤酒館政變', '試圖奪取權力', '但最終失敗並被監禁', '在監禁期間希特勒撰寫了其自傳及政治宣言我的奮鬥的第一冊', '1924年被釋放後', '希特勒對凡爾賽條約進行批判', '宣揚泛日耳曼主義、反猶太主義和反共主義', '以其個人魅力、演說才能及政治宣傳獲得了廣泛的民衆支持', '同時他頻繁宣稱國際資本主義及共產主義爲猶太人反德意志民族的陰謀', '1933年', '納粹黨成爲德國國會第一大黨', '1月30日希特勒被任命爲德國總理', '此後其執政聯盟再次在選舉中獲勝', '並在國會中通過授權法', '逐漸將德國轉變爲一黨專制、納粹、極權及獨裁統治的納粹德國', '希特勒借口抵禦一戰後由英國和法國主導的不公國際秩序', '試圖將猶太人從德國清除', '建立其理想秩序', '在其當政的前六年內', '德國迅速自大蕭條中復甦', '打破一戰後歐洲各國對其作出的各種限制', '吞併數個德意志民族所居住的他國領土', '由此獲得大多數德意志民族人民的支持', '\n', '希特勒試圖爲德意志民族獲取在其他民族之上的特別生存空間', '在外交上主張施行侵略與吞併', '是第二次世界大戰在歐洲爆發的首要原因', '他領導大規模武裝擴軍', '並於1939年9月1日入侵波蘭', '打響第二次世界大戰並使得英國和法國對德國宣

subtitles:
['1936年10月25日德國和義大利達成協調外交政策的同盟條約', '建立柏林－羅馬軸心', '並與日本帝國於1936年11月25日簽署反共產國際協定', '次年義大利於11月6日加入該協定', '\n', '1938年2月希特勒必須解決遠東外交困境', '在中日戰爭中', '選擇持續與中國自1910年代建立的中德關係或與日本結合新聯盟', '在當時的軍隊強烈支持與中華民國持續友好關係', '中德關係獲得外交部長康斯坦丁·馮·紐賴特與戰爭部長維爾納·馮·勃洛姆堡的支持', '他們試著利用外交政策使歐洲遠離戰爭', '但兩人在1938年初期則被希特勒解職', '希特勒任命裏賓特洛甫為新任外交部長', '他支持與日本建立軍事同盟', '繼而希特勒終止了中德關係', '選擇了軍事上較現代化、較強大的日本', '在之後的德國議會中', '希特勒宣佈承認日本所扶持的滿洲國並聲明放棄一戰戰敗在太平洋所失去被日本取得的殖民地', '希特勒命令終止德國一切對中國的物資與人力上的援助', '而中國的國民政府軍事委員會委員長蔣中正取消了中德經濟協定作為報復', '德國失去了自中國而來的軍需原料', '僅能從開放市場取得有限的貨物', '1939年5月22日與義大利簽訂鋼鐵條約', '1940年9月24日在德國柏林', '希特勒、義大利外長加萊阿佐·齊亞諾及日本駐德大使來棲三郎簽署三國同盟條約', '象徵二戰軸心國的確立', '\n', '戰前', '希特勒為改善德波關係', '簽定德波互不侵犯條約', '意圖藉此減弱法國與波蘭的合作關係', '甚至曾想拉波蘭加入反共產國際協定以對抗蘇聯', '但另一方面', '希特勒又想收回一次大戰德國所失去的領土', '並利用德國民族主義解放這些領土', '1938年希特勒想在德國本土與東普魯士之間建立公路', '但遭波蘭拒絕', '在併吞捷克斯洛伐克後', '波蘭人懷疑希特勒的企圖', '另外在1939年3月31日英國反應德國併吞捷克斯洛伐克的行為', '宣佈英法將支持並保證波蘭、比利時、羅馬尼亞、希臘及土耳其的獨立', '4月6日英國與波蘭簽軍事協定', '4月28日希特勒廢除德波互不侵犯條約及英德海軍協定', '1939年初雖然希特勒已計劃用軍事行動來解決與波蘭的領土糾紛', '但爲避免入侵波蘭將蘇聯捲入戰爭並導致德國兩

subtitles:
['時任陸軍總司令瓦爾特·馮·布勞希奇、前任參謀總長路德維希·貝克、參謀總長弗朗茲·哈爾德、維茨勒本、伯魯克道夫及軍需處長、柏林警察局長等人', '他們已約定於1938年9月14日下午8時', '乘希特勒返抵柏林時舉事', '劫持希特勒', '\n', '在他們萬事俱備之後', '忽然於舉事前四小時得知', '英國首相張伯倫、法國總理達拉第已對德軍入侵捷克一案作出綏靖的重大讓步', '希特勒被視爲民族英雄', '這些德國將軍不希望違背民意', '於是終止計畫', '\n', '1944年之後', '盟軍逐漸佔據優勢', '使得一些德國國防軍軍官試圖以刺殺希特勒', '發動政變的方式', '尋求對德國最有利的結局', '1944年7月20日', '希特勒在狼穴召開軍事會議', '克勞斯·馮·施陶芬貝格上校以開會的機會', '偷偷放置了一枚定時炸彈', '是為7月20日密謀案', '炸彈爆炸時', '因巧合未能直接殺死希特勒', '而密謀案也因此失敗', '僥倖逃生的希特勒的神經和聽力因而嚴重受損', '負傷的希特勒發動了大肅清', '超過4900人因涉及此案而被判有罪並遭處決', '參加政變的人包括埃爾溫·馮·維茨萊本元帥、路德維希·貝克上將、埃裏希·赫普納上將、卡爾·格德勒、梅爾茨·馮·基爾海姆上校、阿爾弗雷德·臺爾普神父等等', '埃爾溫·隆美爾元帥和克魯格元帥都被指牽涉其中', '最後兩人被迫自殺']
--------------------------------------------------
著作
subtitles:
Error making subtitles
--------------------------------------------------
相关艺术作品
subtitles:
Error making subtitles
--------------------------------------------------
注释
subtitles:
[]
--------------------------------------------------
參考文獻
subtitles:
['書籍']
--------------------------------------------------
外部链

In [ ]:
# For equation handling

import re

eq_text = sections[6]['content']
print(re.sub(r'(\s){5,}','',eq_text))